In [7]:
# !pip install --upgrade tables
# !pip install eli5
HOME = '/content/drive/My Drive/Colab Notebooks/matrix/dataworkshop_matrix_car'
%cd $HOME

/content/drive/My Drive/Colab Notebooks/matrix/dataworkshop_matrix_car


In [0]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score


import eli5
from eli5.sklearn import PermutationImportance

In [14]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [16]:
df.select_dtypes(np.number).columns

Index(['price_value', 'car_id'], dtype='object')

In [21]:
X = df[ ['car_id'] ]
y = df['price_value']

model = DummyRegressor()
model.fit(X, y)
y_pred = model.predict(X)

mae(y, y_pred)

39465.934630440985

In [23]:
[x for x in df.columns if 'price' in x]

['price_currency', 'price_details', 'price_value']

In [29]:
df.price_currency.value_counts(normalize=True)

PLN    0.998084
EUR    0.001916
Name: price_currency, dtype: float64

In [30]:
df = df[ df['price_currency'] != 'EUR']
df.price_currency.value_counts(normalize=True)

PLN    1.0
Name: price_currency, dtype: float64

In [37]:
SUFFIX = ' __cat'

for feat in df.columns:
    if isinstance(df[feat][0], list):
        continue

    factorized_feat = df[feat].factorize()[0]
    if SUFFIX in feat:
        df[feat] = factorized_feat
    else:
        df[feat + SUFFIX] = factorized_feat

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [41]:
cat_feats = [x for x in df.columns if SUFFIX in x and 'price' not in x]
len(cat_feats)

151

In [46]:
X = df[cat_feats].values
y = df['price_value'].values

model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
np.mean(scores)

-19566.588937368328

In [49]:
new_model = DecisionTreeRegressor(max_depth=5)
new_model.fit(X, y)

imp = PermutationImportance(new_model, random_state=0).fit(X, y)
eli5.show_weights(imp, feature_names=cat_feats)

Weight,Feature
0.2533 ± 0.0040,param_napęd __cat
0.2008 ± 0.0047,param_faktura-vat __cat
0.1943 ± 0.0088,param_stan __cat
0.1423 ± 0.0086,param_rok-produkcji __cat
0.0629 ± 0.0047,param_moc __cat
0.0424 ± 0.0014,feature_kamera-cofania __cat
0.0412 ± 0.0008,param_skrzynia-biegów __cat
0.0286 ± 0.0037,param_marka-pojazdu __cat
0.0191 ± 0.0022,param_pojemność-skokowa __cat
0.0163 ± 0.0005,feature_bluetooth __cat
